This notebook implements the complete system for:

Activity 1: PIN code entry (4 digits: 0–9)
Activity 2: Application detection

Architecture:
    DT + SVM: Determines the activity (PIN or Application) from 19 features
    PIN CNN: If the activity = PIN, classifies which digit (0–9)
    Application CNN: If the activity = Application, classifies which app

--------------------------
Imports and GPU configuration
-

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Sklearn pour DT + SVM
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

# SciPy pour le prétraitement des signaux
from scipy import signal as scipy_signal
from scipy.stats import entropy as scipy_entropy

# Librosa pour MFCC
import librosa

# TensorFlow/Keras pour CNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# Configuration matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("="*70)
print("                    CONFIGURATION SYSTÈME")
print("="*70)
print(f"TensorFlow version: {tf.__version__}")
print(f"Nombre de GPUs: {len(tf.config.list_physical_devices('GPU'))}")

# Configuration GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU configuré: {gpus[0].name}")
    except RuntimeError as e:
        print(f"❌ Erreur GPU: {e}")
else:
    print("⚠️  CPU utilisé (pas de GPU détecté)")

print("="*70)

2026-01-19 10:54:46.067251: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-19 10:54:47.361002: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-19 10:54:51.011948: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


                    CONFIGURATION SYSTÈME
TensorFlow version: 2.20.0
Nombre de GPUs: 1
✅ GPU configuré: /physical_device:GPU:0


-------------------
Data pre-treatement functions
-

In [3]:
print("\n" + "="*70)
print("     DÉFINITION DES FONCTIONS DE PRÉTRAITEMENT")
print("="*70)

def apply_lowpass_filter(raw_signal, sampling_freq=5000, cutoff_freq=500):
    """
    Applique un filtre passe-bas Butterworth d'ordre 4
    Selon Section 5.2 du papier
    
    Args:
        raw_signal: signal brut
        sampling_freq: fréquence d'échantillonnage (5 KHz par défaut)
        cutoff_freq: fréquence de coupure (500 Hz)
    """
    nyquist = sampling_freq / 2
    normalized_cutoff = cutoff_freq / nyquist
    
    # Filtre Butterworth d'ordre 4
    b, a = scipy_signal.butter(4, normalized_cutoff, btype='low')
    filtered_signal = scipy_signal.filtfilt(b, a, raw_signal)
    
    return filtered_signal


def calculate_sliding_window_sums(signal_data, window_size=50):
    """
    Calcule les sommes dans une fenêtre glissante
    Implémente l'équation (14) du papier:
    S = sum(|v_{i+1} - v_i|) pour i dans la fenêtre
    
    Args:
        signal_data: signal filtré
        window_size: taille de la fenêtre (Nw dans le papier)
    
    Returns:
        array des sommes
    """
    sums = []
    for i in range(len(signal_data) - window_size):
        window = signal_data[i:i+window_size]
        S = np.sum(np.abs(np.diff(window)))  # Eq. (14)
        sums.append(S)
    
    return np.array(sums)


def detect_activity_boundaries(sums, n_consecutive=5):
    """
    Détecte les points de début et fin des activités
    
    Args:
        sums: array des sommes de la fenêtre glissante
        n_consecutive: nombre de points consécutifs pour confirmer début/fin
    
    Returns:
        list de tuples (start_idx, end_idx)
    """
    # Calculer le seuil
    threshold = np.mean(sums) + 2 * np.std(sums)
    
    boundaries = []
    i = 0
    
    while i < len(sums) - n_consecutive:
        # Chercher un début d'activité (n points consécutifs en hausse)
        if all(sums[i+j] < sums[i+j+1] for j in range(n_consecutive-1)):
            start = i
            
            # Chercher la fin (n points consécutifs en baisse)
            j = start + n_consecutive
            while j < len(sums) - n_consecutive:
                if all(sums[j+k] > sums[j+k+1] for k in range(n_consecutive-1)):
                    end = j + n_consecutive
                    boundaries.append((start, end))
                    i = end
                    break
                j += 1
            
            if j >= len(sums) - n_consecutive:
                # Pas de fin trouvée, prendre jusqu'à la fin
                boundaries.append((start, len(sums)))
                break
        
        i += 1
    
    return boundaries


def preprocess_voltage_signal(raw_signal, sampling_freq=5000, cutoff_freq=500, 
                               window_size=50, n_consecutive=5):
    """
    Prétraitement complet selon Section 5.2 du papier
    
    Étapes:
    1. Filtrage passe-bas (500 Hz)
    2. Détection d'activité avec fenêtre glissante (Eq. 14)
    3. Segmentation des activités
    4. Normalisation (soustraire le minimum)
    
    Args:
        raw_signal: signal de voltage brut
        sampling_freq: fréquence d'échantillonnage (Hz)
        cutoff_freq: fréquence de coupure du filtre (Hz)
        window_size: taille de la fenêtre glissante
        n_consecutive: nombre de points consécutifs pour détecter activité
    
    Returns:
        list de segments normalisés
    """
    # Étape 1: Filtrage passe-bas
    filtered = apply_lowpass_filter(raw_signal, sampling_freq, cutoff_freq)
    
    # Étape 2: Calcul des sommes avec fenêtre glissante
    sums = calculate_sliding_window_sums(filtered, window_size)
    
    # Étape 3: Détection des frontières d'activités
    boundaries = detect_activity_boundaries(sums, n_consecutive)
    
    # Segmentation et normalisation
    segments = []
    for start, end in boundaries:
        segment = filtered[start:end]
        
        # Étape 4: Normalisation (soustraire le minimum)
        normalized = segment - np.min(segment)
        segments.append(normalized)
    
    return segments


def extract_19_features(signal_segment, sampling_freq=5000):
    """
    Extrait exactement 19 features comme dans le papier (Section 5.2)
    
    Features extraites:
    - 6 features statistiques
    - 13 MFCC (Mel Frequency Cepstral Coefficients)
    
    Args:
        signal_segment: segment de signal normalisé
        sampling_freq: fréquence d'échantillonnage
    
    Returns:
        array de 19 features
    """
    features = []
    
    # ===== 6 Features Statistiques =====
    features.append(np.max(signal_segment))                    # 1. Maximum
    features.append(np.std(signal_segment))                    # 2. Standard deviation
    features.append(np.var(signal_segment))                    # 3. Variance
    features.append(np.mean(signal_segment))                   # 4. Mean
    features.append(scipy_entropy(signal_segment + 1e-10))     # 5. Entropy
    features.append(len(signal_segment))                       # 6. Length (durée)
    
    # ===== 13 MFCC =====
    # Convertir en float et calculer les MFCC
    signal_float = signal_segment.astype(np.float32)
    
    # Calculer 13 MFCC
    mfccs = librosa.feature.mfcc(
        y=signal_float,
        sr=sampling_freq,
        n_mfcc=13
    )
    
    # Prendre la moyenne de chaque coefficient MFCC
    mfcc_means = np.mean(mfccs, axis=1)
    features.extend(mfcc_means)
    
    return np.array(features)


print("✅ Fonctions de prétraitement définies:")
print("   - apply_lowpass_filter() : Filtre passe-bas 500 Hz")
print("   - calculate_sliding_window_sums() : Eq. (14) du papier")
print("   - detect_activity_boundaries() : Détection début/fin")
print("   - preprocess_voltage_signal() : Pipeline complet")
print("   - extract_19_features() : 6 stat + 13 MFCC")
print("="*70)


     DÉFINITION DES FONCTIONS DE PRÉTRAITEMENT
✅ Fonctions de prétraitement définies:
   - apply_lowpass_filter() : Filtre passe-bas 500 Hz
   - calculate_sliding_window_sums() : Eq. (14) du papier
   - detect_activity_boundaries() : Détection début/fin
   - preprocess_voltage_signal() : Pipeline complet
   - extract_19_features() : 6 stat + 13 MFCC


-----------------------------------------
Charge Data and prepare them
-

In [ ]:
"""
FORMAT DES DONNÉES ATTENDU :

Pour chaque échantillon, vous devez avoir :
1. raw_voltage : vecteur de N valeurs (voltages bruts) - sera prétraité
2. activity : 0 = PIN, 1 = Application
3. label : 
   - Si activity=0 (PIN) : chiffre du PIN (0-9)
   - Si activity=1 (App) : ID de l'application (0, 1, 2, ...)

PARAMÈTRES DE COLLECTE (selon le papier):
- Fréquence d'échantillonnage : 5 KHz
- État de la batterie : 20-30%
- Résistance : 0.33 Ohm sur la ligne d'alimentation
"""

# ════════════════════════════════════════════════════════════════
# OPTION 1 : DONNÉES SYNTHÉTIQUES (pour tester le système)
# ════════════════════════════════════════════════════════════════

def generate_synthetic_voltage_data(n_samples=2000, base_length=1000, sampling_freq=5000):
    """
    Génère des données de voltage synthétiques réalistes
    
    Args:
        n_samples: nombre total d'échantillons
        base_length: longueur de base des signaux
        sampling_freq: fréquence d'échantillonnage
    
    Returns:
        raw_voltages, y_activity, y_class
    """
    print("🔄 Génération de données synthétiques...")
    
    # Moitié PIN, moitié Applications
    n_pin = n_samples // 2
    n_app = n_samples - n_pin
    
    # Générer des signaux de voltage bruts (avec bruit et activités)
    raw_voltages = []
    y_activity = []
    y_class = []
    
    # Signaux PIN (chiffres 0-9)
    for i in range(n_pin):
        # Bruit de fond
        length = np.random.randint(base_length - 200, base_length + 200)
        noise = np.random.randn(length) * 0.1
        
        # Activité (pic de voltage)
        digit = np.random.randint(0, 10)
        activity_pos = length // 2
        activity_width = 100
        
        # Créer un pic caractéristique
        for j in range(activity_width):
            pos = activity_pos + j
            if pos < length:
                noise[pos] += np.sin(j / activity_width * np.pi) * (digit + 1) * 0.5
        
        raw_voltages.append(noise)
        y_activity.append(0)  # PIN
        y_class.append(digit)
    
    # Signaux Application (apps 0-4)
    for i in range(n_app):
        # Bruit de fond
        length = np.random.randint(base_length - 200, base_length + 200)
        noise = np.random.randn(length) * 0.15
        
        # Activité (signature de l'app)
        app_id = np.random.randint(0, 5)
        activity_pos = length // 2
        activity_width = 150
        
        # Créer une signature différente pour chaque app
        for j in range(activity_width):
            pos = activity_pos + j
            if pos < length:
                noise[pos] += np.cos(j / activity_width * np.pi * (app_id + 1)) * 0.7
        
        raw_voltages.append(noise)
        y_activity.append(1)  # Application
        y_class.append(app_id)
    
    # Mélanger
    indices = np.random.permutation(n_samples)
    raw_voltages = [raw_voltages[i] for i in indices]
    y_activity = np.array([y_activity[i] for i in indices])
    y_class = np.array([y_class[i] for i in indices])
    
    print(f"✅ Données générées:")
    print(f"   Total échantillons: {n_samples}")
    print(f"   PIN samples: {(y_activity==0).sum()}")
    print(f"   App samples: {(y_activity==1).sum()}")
    
    return raw_voltages, y_activity, y_class


# Générer les données
raw_voltages, y_activity, y_class = generate_synthetic_voltage_data(
    n_samples=2000,
    base_length=1000,
    sampling_freq=5000
)

print(f"\n📊 Données brutes générées: {len(raw_voltages)} échantillons")

----------------------
Pre-treatment and features extraction
-

In [ ]:
print("\n" + "="*70)
print("     PRÉTRAITEMENT DES SIGNAUX ET EXTRACTION DES FEATURES")
print("="*70)

SAMPLING_FREQ = 5000  # Hz

# Conteneurs pour les données traitées
X_features_list = []  # Features pour DT+SVM
X_sequences_list = []  # Séquences pour CNN
y_activity_processed = []
y_class_processed = []

print(f"\n🔄 Traitement de {len(raw_voltages)} signaux bruts...")

for idx, raw_signal in enumerate(raw_voltages):
    if (idx + 1) % 200 == 0:
        print(f"   Traité: {idx + 1}/{len(raw_voltages)}")
    
    # Prétraiter le signal (filtrage + détection + segmentation)
    segments = preprocess_voltage_signal(
        raw_signal,
        sampling_freq=SAMPLING_FREQ,
        cutoff_freq=500,
        window_size=50,
        n_consecutive=5
    )
    
    # Pour chaque segment détecté
    for segment in segments:
        if len(segment) < 50:  # Ignorer les segments trop courts
            continue
        
        # Extraire les 19 features
        features = extract_19_features(segment, SAMPLING_FREQ)
        
        # Normaliser la séquence pour le CNN (Z-score)
        mean = np.mean(segment)
        std = np.std(segment) + 1e-8
        sequence_normalized = (segment - mean) / std
        
        # Stocker
        X_features_list.append(features)
        X_sequences_list.append(sequence_normalized)
        y_activity_processed.append(y_activity[idx])
        y_class_processed.append(y_class[idx])

# Convertir en arrays
X_features = np.array(X_features_list)
y_activity = np.array(y_activity_processed)
y_class = np.array(y_class_processed)

# Pour les séquences, on doit les padding à la même longueur
max_length = max(len(seq) for seq in X_sequences_list)
print(f"\n📐 Longueur maximale des séquences: {max_length}")

# Padding ou truncation
X_sequences = []
for seq in X_sequences_list:
    if len(seq) < max_length:
        # Padding avec des zéros
        padded = np.pad(seq, (0, max_length - len(seq)), mode='constant')
    else:
        # Truncation
        padded = seq[:max_length]
    X_sequences.append(padded)

X_sequences = np.array(X_sequences)

print(f"\n✅ Prétraitement terminé:")
print(f"   Features shape: {X_features.shape} (19 features par échantillon)")
print(f"   Sequences shape: {X_sequences.shape}")
print(f"   Activity labels: {y_activity.shape}")
print(f"   Class labels: {y_class.shape}")
print(f"   PIN samples: {(y_activity==0).sum()}")
print(f"   App samples: {(y_activity==1).sum()}")

print("="*70)

--------------
Exploration of treated Data
-

In [ ]:
print("\n" + "="*70)
print("                    EXPLORATION DES DONNÉES")
print("="*70)

print(f"\n📊 Statistiques générales:")
print(f"   Nombre total d'échantillons: {len(X_features)}")
print(f"   Dimension des features: {X_features.shape[1]} (devrait être 19)")
print(f"   Longueur des séquences: {X_sequences.shape[1]}")

print(f"\n🎯 Distribution des activités:")
unique_activities, counts_activities = np.unique(y_activity, return_counts=True)
for act, count in zip(unique_activities, counts_activities):
    activity_name = "PIN" if act == 0 else "Application"
    print(f"   {activity_name}: {count} échantillons ({count/len(y_activity)*100:.1f}%)")

print(f"\n🔢 Distribution pour PIN (activité=0):")
pin_mask = (y_activity == 0)
pin_classes = y_class[pin_mask]
unique_pins, counts_pins = np.unique(pin_classes, return_counts=True)
print(f"   Classes PIN détectées: {unique_pins}")
for pin, count in zip(unique_pins, counts_pins):
    print(f"      Chiffre {pin}: {count} échantillons")

print(f"\n📱 Distribution pour Applications (activité=1):")
app_mask = (y_activity == 1)
app_classes = y_class[app_mask]
unique_apps, counts_apps = np.unique(app_classes, return_counts=True)
print(f"   Applications détectées: {unique_apps}")
for app_id, count in zip(unique_apps, counts_apps):
    print(f"      App {app_id}: {count} échantillons")

# Visualisation des 19 features
print(f"\n📈 Statistiques des features (19 dimensions):")
feature_names = ['Max', 'Std', 'Var', 'Mean', 'Entropy', 'Length'] + \
                [f'MFCC{i+1}' for i in range(13)]

for i, name in enumerate(feature_names):
    print(f"   {name:10s}: min={X_features[:, i].min():.3f}, " +
          f"max={X_features[:, i].max():.3f}, " +
          f"mean={X_features[:, i].mean():.3f}")

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Distribution activités
axes[0, 0].bar(['PIN', 'Application'], counts_activities, color=['#3498db', '#e74c3c'])
axes[0, 0].set_title('Distribution des Activités', fontweight='bold', fontsize=12)
axes[0, 0].set_ylabel('Nombre d\'échantillons')
for i, (act, count) in enumerate(zip(['PIN', 'App'], counts_activities)):
    axes[0, 0].text(i, count, str(count), ha='center', va='bottom', fontweight='bold')

# Distribution PIN
axes[0, 1].bar(unique_pins.astype(str), counts_pins, color='#3498db')
axes[0, 1].set_title('Distribution des Chiffres PIN', fontweight='bold', fontsize=12)
axes[0, 1].set_xlabel('Chiffre')
axes[0, 1].set_ylabel('Nombre d\'échantillons')

# Distribution Apps
axes[1, 0].bar([f'App {i}' for i in unique_apps], counts_apps, color='#e74c3c')
axes[1, 0].set_title('Distribution des Applications', fontweight='bold', fontsize=12)
axes[1, 0].set_xlabel('Application')
axes[1, 0].set_ylabel('Nombre d\'échantillons')

# Exemple de séquences
axes[1, 1].plot(X_sequences[0], label='PIN exemple', alpha=0.7)
axes[1, 1].plot(X_sequences[len(X_sequences)//2], label='App exemple', alpha=0.7)
axes[1, 1].set_title('Exemples de Séquences Normalisées', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Temps')
axes[1, 1].set_ylabel('Voltage normalisé')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data_exploration.png', dpi=300, bbox_inches='tight')
plt.show()

print("="*70)

----------------
Creation of Decision tree + SVM toanalyse activity
-

In [ ]:
print("\n" + "="*70)
print("     ÉTAPE 1/5 : CRÉATION DU CLASSIFICATEUR DT + SVM")
print("="*70)

class ActivityClassifier:
    """
    Classificateur d'activité : Decision Tree + SVM en Voting
    Détermine si l'échantillon est un PIN (0) ou une Application (1)
    Utilise 19 features extraites des signaux
    """
    
    def __init__(self):
        self.scaler = StandardScaler()
        self.dt_classifier = None
        self.svm_classifier = None
        self.voting_classifier = None
        self.is_trained = False
        
    def create(self):
        """Crée le classificateur combiné"""
        print("\n🔨 Création du classificateur...")
        
        # Decision Tree
        self.dt_classifier = DecisionTreeClassifier(
            max_depth=10,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42
        )
        print("   ✅ Decision Tree créé (max_depth=10)")
        
        # SVM avec kernel RBF
        self.svm_classifier = SVC(
            kernel='rbf',
            C=1.0,
            gamma='scale',
            probability=True,
            random_state=42
        )
        print("   ✅ SVM créé (kernel=rbf)")
        
        # Voting Classifier (soft voting)
        self.voting_classifier = VotingClassifier(
            estimators=[
                ('decision_tree', self.dt_classifier),
                ('svm', self.svm_classifier)
            ],
            voting='soft'  # Utilise les probabilités
        )
        print("   ✅ Voting Classifier créé (soft voting)")
        
        return self

# Créer le classificateur
activity_classifier = ActivityClassifier()
activity_classifier.create()

print("\n✅ Classificateur d'activité créé avec succès!")
print("="*70)

----------------
Training of DT + SVM
-

In [ ]:
print("\n" + "="*70)
print("     ÉTAPE 2/5 : ENTRAÎNEMENT DU DT + SVM")
print("="*70)

def train_activity_classifier(classifier, X_features, y_activity, test_size=0.25):
    """
    Entraîne le classificateur d'activité
    Utilise 75% pour l'entraînement, 25% pour le test (selon le papier)
    """
    
    print("\n📊 Préparation des données...")
    
    # Split train/test (75/25 selon le papier)
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, y_activity, 
        test_size=test_size, 
        random_state=42, 
        stratify=y_activity
    )
    
    print(f"   Train: {X_train.shape[0]} échantillons (75%)")
    print(f"   Test:  {X_test.shape[0]} échantillons (25%)")
    
    # Normalisation des features (important pour SVM)
    print("\n🔄 Normalisation des 19 features...")
    X_train_scaled = classifier.scaler.fit_transform(X_train)
    X_test_scaled = classifier.scaler.transform(X_test)
    
    # Entraînement
    print("\n🚀 Entraînement en cours...")
    classifier.voting_classifier.fit(X_train_scaled, y_train)
    classifier.is_trained = True
    
    # Évaluation
    print("\n📈 Évaluation...")
    train_score = classifier.voting_classifier.score(X_train_scaled, y_train)
    test_score = classifier.voting_classifier.score(X_test_scaled, y_test)
    
    print(f"\n✅ Résultats:")
    print(f"   Train Accuracy: {train_score:.4f} ({train_score*100:.2f}%)")
    print(f"   Test Accuracy:  {test_score:.4f} ({test_score*100:.2f}%)")
    
    # Prédictions détaillées
    y_pred = classifier.voting_classifier.predict(X_test_scaled)
    
    print("\n📊 Rapport de classification:")
    print(classification_report(y_test, y_pred, 
                               target_names=['PIN', 'Application'],
                               digits=4))
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['PIN', 'Application'],
                yticklabels=['PIN', 'Application'],
                cbar_kws={'label': 'Nombre d\'échantillons'})
    plt.title('Matrice de Confusion - DT+SVM (Activité)', 
             fontweight='bold', fontsize=14)
    plt.ylabel('Vraie Activité')
    plt.xlabel('Activité Prédite')
    plt.tight_layout()
    plt.savefig('confusion_matrix_activity.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return train_score, test_score

# Entraîner
train_acc, test_acc = train_activity_classifier(
    activity_classifier, 
    X_features, 
    y_activity,
    test_size=0.25  # 75/25 split selon le papier
)

print("\n✅ DT + SVM entraîné avec succès!")
print("="*70)

---------------------------------------------------
Definition of modele
---------------------------------------------------

In [2]:
def create_cnn_model(input_shape, num_classes):
    """
    Crée le modèle CNN 1D selon l'architecture du schéma:
    - C-16 KS-8 S-4: Conv1D 16 filtres, kernel_size=8, stride=4
    - C-32 KS-8 S-4: Conv1D 32 filtres, kernel_size=8, stride=4
    - C-64 KS-8 S-4: Conv1D 64 filtres, kernel_size=8, stride=4
    - Chaque conv suivie de BatchNorm + ReLU
    - Flatten + Dense + Output
    """
    model = models.Sequential([
        # Input
        layers.Input(shape=input_shape, name='input'),
        
        # Bloc 1: C-16 KS-8 S-4
        layers.Conv1D(filters=16, kernel_size=8, strides=4, padding='same', name='conv1'),
        layers.BatchNormalization(name='bn1'),
        layers.ReLU(name='relu1'),
        
        # Bloc 2: C-32 KS-8 S-4
        layers.Conv1D(filters=32, kernel_size=8, strides=4, padding='same', name='conv2'),
        layers.BatchNormalization(name='bn2'),
        layers.ReLU(name='relu2'),
        
        # Bloc 3: C-64 KS-8 S-4
        layers.Conv1D(filters=64, kernel_size=8, strides=4, padding='same', name='conv3'),
        layers.BatchNormalization(name='bn3'),
        layers.ReLU(name='relu3'),
        
        # Flatten
        layers.Flatten(name='flatten'),
        
        # First fully-connected layer
        layers.Dense(128, name='fc1'),
        layers.Dropout(0.5, name='dropout1'),
        
        # Sigmoid activation pour augmenter la non-linéarité (selon le papier)
        layers.Activation('sigmoid', name='sigmoid'),
        
        # Second fully-connected layer
        layers.Dense(64, name='fc2'),
        layers.Dropout(0.3, name='dropout2'),
        
        # Output layer
        layers.Dense(num_classes, activation='softmax', name='output')
    ], name='CNN_Keystroke_Model')
    
    return model

print("✅ Fonction create_cnn_model définie")

✅ Fonction create_cnn_model définie


-------------------------------------------------------------------
Utils Functions
-------------------------------------------------------------------

In [3]:
def prepare_data(X, y, test_size=0.2, val_size=0.1, normalize=True):
    """
    Prépare les données pour l'entraînement.
    
    Args:
        X: array (n_samples, sequence_length) - séquences de volts
        y: array (n_samples,) - labels (digits)
        test_size: proportion du test set
        val_size: proportion du validation set (par rapport au train)
        normalize: normaliser les séquences
    
    Returns:
        X_train, X_val, X_test, y_train, y_val, y_test
    """
    print("\n" + "="*60)
    print("Préparation des données")
    print("="*60)
    
    # Normalisation (Z-score normalization)       ---------------------------------------------------------------------
    if normalize:
        print("📊 Normalisation des données...")
        mean = np.mean(X, axis=1, keepdims=True)
        std = np.std(X, axis=1, keepdims=True) + 1e-8
        X = (X - mean) / std
    
    # Reshape pour Conv1D: (n_samples, sequence_length, 1)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    print(f"📐 Shape après reshape: {X.shape}")
    
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )
    
    # Split train/validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=42, stratify=y_train
    )
    
    print(f"✅ Train set: {X_train.shape[0]} samples")
    print(f"✅ Val set:   {X_val.shape[0]} samples")
    print(f"✅ Test set:  {X_test.shape[0]} samples")
    print(f"✅ Classes:   {len(np.unique(y))} ({np.unique(y)})")
    print("="*60)
    
    return X_train, X_val, X_test, y_train, y_val, y_test


def plot_training_history(history, save_path='training_history.png'):
    """Visualise l'historique d'entraînement"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Loss
    axes[1].plot(history.history['loss'], label='Train', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"📊 Graphique sauvegardé: {save_path}")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, classes, save_path='confusion_matrix.png'):
    """Affiche la matrice de confusion"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes,
                cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"📊 Matrice de confusion sauvegardée: {save_path}")
    plt.show()


print("✅ Fonctions utilitaires définies")

✅ Fonctions utilitaires définies


------------------------------------------------------------------------
Charge Data
------------------------------------------------------------------------

In [5]:
def load_data_from_csv(file_path):
    """
    Charge les données depuis un fichier CSV
    Format attendu: première colonne = digit, colonnes suivantes = valeurs de volt
    """
    print(f"📂 Chargement depuis: {file_path}")
    df = pd.read_csv(file_path)
    
    # Extraire les labels (première colonne)
    y = df.iloc[:, 0].values
    
    # Extraire les features (colonnes restantes)
    X = df.iloc[:, 1:].values
    
    print(f"✅ Données chargées: X={X.shape}, y={y.shape}")
    print(f"   Classes trouvées: {np.unique(y)}")
    
    return X, y

# # Charger vos données
# X, y = load_data_from_csv('/mnt/c/Users/VotreNom/Documents/votre_fichier.csv')

-------------------------------------------------------------------------------
Data exploration
-------------------------------------------------------------------------------

In [ ]:
print("\n" + "="*60)
print("Exploration des données")
print("="*60)

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"Type de X: {X.dtype}")
print(f"Type de y: {y.dtype}")
print(f"\nNombre de classes: {len(np.unique(y))}")
print(f"Classes: {np.unique(y)}")
print(f"\nDistribution des classes:")
unique, counts = np.unique(y, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"  Classe {cls}: {count} samples ({count/len(y)*100:.1f}%)")

print(f"\nStatistiques des séquences:")
print(f"  Min: {X.min():.4f}")
print(f"  Max: {X.max():.4f}")
print(f"  Mean: {X.mean():.4f}")
print(f"  Std: {X.std():.4f}")

# Visualiser quelques séquences
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for i in range(6):
    idx = np.random.randint(0, len(X))
    axes[i].plot(X[idx], linewidth=0.5)
    axes[i].set_title(f'Sample {idx} - Classe {y[idx]}')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel('Voltage')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('sample_sequences.png', dpi=300, bbox_inches='tight')
plt.show()

print("="*60)

-------------------------------------------------------------------------------------------
Data preparation
-------------------------------------------------------------------------------------------

In [ ]:
# Préparer les données
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(
    X, y, 
    test_size=0.2, 
    val_size=0.1,
    normalize=True
)

# Vérifier les shapes
print(f"\n📐 Shapes finales:")
print(f"   X_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"   X_val:   {X_val.shape}   | y_val:   {y_val.shape}")
print(f"   X_test:  {X_test.shape}  | y_test:  {y_test.shape}")

--------------------------------------------------------------------------------
Creation and compilation of the CNN model
--------------------------------------------------------------------------------

In [ ]:
# Paramètres du modèle
input_shape = (X_train.shape[1], 1)  # (sequence_length, 1)
num_classes = len(np.unique(y))

print("\n" + "="*60)
print("Création du modèle")
print("="*60)
print(f"Input shape: {input_shape}")
print(f"Nombre de classes: {num_classes}")

# Créer le modèle
model = create_cnn_model(input_shape, num_classes)

# Afficher l'architecture
model.summary()

# Compiler le modèle
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✅ Modèle compilé et prêt pour l'entraînement")
print("="*60)

-------------------------------------------------------------------------------
Callback Configuration
-------------------------------------------------------------------------------

In [ ]:
# Créer un timestamp pour les fichiers
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"keystroke_cnn_{timestamp}"

# Callbacks
callbacks = [
    # Early Stopping: arrête si pas d'amélioration
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce Learning Rate: réduit le LR si plateau
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Model Checkpoint: sauvegarde le meilleur modèle
    keras.callbacks.ModelCheckpoint(
        f'{model_name}_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    # TensorBoard: pour visualiser l'entraînement
    keras.callbacks.TensorBoard(
        log_dir=f'./logs/{model_name}',
        histogram_freq=1
    )
]


# Callback personnalisé pour surveiller l'overfitting en temps réel
class OverfittingMonitor(keras.callbacks.Callback):
    """Surveille l'overfitting pendant l'entraînement"""
    def __init__(self, patience=3):
        super().__init__()
        self.patience = patience
        self.wait = 0
        self.best_val_loss = float('inf')
        
    def on_epoch_end(self, epoch, logs=None):
        train_loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        train_acc = logs.get('accuracy')
        val_acc = logs.get('val_accuracy')
        
        # Calculer les écarts
        loss_gap = abs(val_loss - train_loss)
        acc_gap = abs(train_acc - val_acc)
        
        # Vérifier l'amélioration
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.wait = 0
        else:
            self.wait += 1
        
        # Alertes d'overfitting
        if loss_gap > 0.3 or acc_gap > 0.1:
            print(f"\n  ❌ ALERTE: Overfitting sévère à l'epoch {epoch+1}!")
            print(f"     Loss gap: {loss_gap:.4f} | Acc gap: {acc_gap:.4f}")
        elif loss_gap > 0.15 or acc_gap > 0.05:
            print(f"\n  ⚠️  Overfitting modéré à l'epoch {epoch+1}")
            print(f"     Loss gap: {loss_gap:.4f} | Acc gap: {acc_gap:.4f}")


print("✅ Callbacks configurés")
print(f"📁 Modèle sera sauvegardé: {model_name}_best.keras")
print(f"📊 Logs TensorBoard: ./logs/{model_name}")

--------------------------------------------------------------------------------
Model Training
--------------------------------------------------------------------------------

In [ ]:
print("\n" + "="*60)
print("DÉBUT DE L'ENTRAÎNEMENT")
print("="*60)

# Hyperparamètres
BATCH_SIZE = 32
EPOCHS = 50

print(f"Batch size: {BATCH_SIZE}")
print(f"Epochs max: {EPOCHS}")
print("\n🚀 Entraînement en cours...")

# Entraînement
history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("✅ ENTRAÎNEMENT TERMINÉ")
print("="*60)

--------------------------------------------------------------------------
Final Overfitting Analysis
---------------------------------------------------------------------------

In [ ]:
def detect_overfitting(history):
    """Détecte et analyse l'overfitting"""
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    # Métriques finales
    final_train_loss = train_loss[-1]
    final_val_loss = val_loss[-1]
    final_train_acc = train_acc[-1]
    final_val_acc = val_acc[-1]
    
    loss_gap = abs(final_val_loss - final_train_loss)
    acc_gap = abs(final_train_acc - final_val_acc)
    
    print("\n" + "="*70)
    print("🔍 ANALYSE DE L'OVERFITTING")
    print("="*70)
    
    print(f"\n📊 Métriques finales:")
    print(f"   Train Loss:     {final_train_loss:.4f}")
    print(f"   Val Loss:       {final_val_loss:.4f}")
    print(f"   Écart Loss:     {loss_gap:.4f}")
    print(f"   Train Accuracy: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
    print(f"   Val Accuracy:   {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
    print(f"   Écart Accuracy: {acc_gap:.4f} ({acc_gap*100:.2f}%)")
    
    # Diagnostic
    print(f"\n🎯 Diagnostic:")
    if loss_gap > 0.3 or acc_gap > 0.1:
        print("   ❌ OVERFITTING SÉVÈRE détecté !")
        print(f"\n💡 Recommandations:")
        print("   → Augmenter le Dropout à 0.6-0.7")
        print("   → Réduire la complexité du modèle")
        print("   → Ajouter plus de données d'entraînement")
        print("   → Utiliser de la régularisation L2")
    elif loss_gap > 0.15 or acc_gap > 0.05:
        print("   ⚠️  OVERFITTING MODÉRÉ détecté")
        print(f"\n💡 Recommandations:")
        print("   → Augmenter légèrement le Dropout")
        print("   → Utiliser Early Stopping plus agressif (patience=5)")
        print("   → Ajouter du Data Augmentation")
    else:
        print("   ✅ PAS D'OVERFITTING - Le modèle généralise bien !")
    
    print("="*70)
    
    return loss_gap, acc_gap

# Analyser l'overfitting
loss_gap, acc_gap = detect_overfitting(history)

--------------------------------------------------------------------------------------------
Detailled Overfitting Visualisation
--------------------------------------------------------------------------------------------

In [ ]:
def plot_overfitting_analysis(history, save_path='overfitting_analysis.png'):
    """Crée une visualisation complète pour analyser l'overfitting"""
    fig = plt.figure(figsize=(18, 10))
    
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(1, len(train_loss) + 1)
    
    # Calculer les écarts
    loss_gaps = [abs(v - t) for v, t in zip(val_loss, train_loss)]
    acc_gaps = [abs(v - t) for v, t in zip(val_acc, train_acc)]
    
    # 1. Loss Comparison
    plt.subplot(2, 3, 1)
    plt.plot(epochs, train_loss, 'b-', linewidth=2, label='Train Loss')
    plt.plot(epochs, val_loss, 'r-', linewidth=2, label='Val Loss')
    plt.title('Loss Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 2. Accuracy Comparison
    plt.subplot(2, 3, 2)
    plt.plot(epochs, train_acc, 'b-', linewidth=2, label='Train Acc')
    plt.plot(epochs, val_acc, 'r-', linewidth=2, label='Val Acc')
    plt.title('Accuracy Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 3. Loss Gap
    plt.subplot(2, 3, 3)
    plt.plot(epochs, loss_gaps, 'purple', linewidth=2)
    plt.axhline(y=0.15, color='orange', linestyle='--', label='Moderate')
    plt.axhline(y=0.3, color='red', linestyle='--', label='Severe')
    plt.title('Loss Gap (|Val - Train|)', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Gap')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 4. Accuracy Gap
    plt.subplot(2, 3, 4)
    plt.plot(epochs, acc_gaps, 'green', linewidth=2)
    plt.axhline(y=0.05, color='orange', linestyle='--', label='Moderate')
    plt.axhline(y=0.1, color='red', linestyle='--', label='Severe')
    plt.title('Accuracy Gap (|Val - Train|)', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Gap')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 5. Val/Train Loss Ratio
    plt.subplot(2, 3, 5)
    loss_ratios = [v/t if t > 0 else 1 for v, t in zip(val_loss, train_loss)]
    plt.plot(epochs, loss_ratios, 'brown', linewidth=2)
    plt.axhline(y=1.0, color='green', linestyle='--', label='Perfect')
    plt.axhline(y=1.2, color='orange', linestyle='--', label='Moderate')
    plt.axhline(y=1.5, color='red', linestyle='--', label='Severe')
    plt.title('Val/Train Loss Ratio', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Ratio')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 6. Diagnostic Final
    plt.subplot(2, 3, 6)
    plt.axis('off')
    
    final_loss_gap = loss_gaps[-1]
    final_acc_gap = acc_gaps[-1]
    
    if final_loss_gap > 0.3 or final_acc_gap > 0.1:
        status = "❌ OVERFITTING\nSÉVÈRE"
        color = 'red'
    elif final_loss_gap > 0.15 or final_acc_gap > 0.05:
        status = "⚠️  OVERFITTING\nMODÉRÉ"
        color = 'orange'
    else:
        status = "✅ BON\nAJUSTEMENT"
        color = 'green'
    
    text = f"""
DIAGNOSTIC FINAL

{status}

Train Loss: {train_loss[-1]:.4f}
Val Loss:   {val_loss[-1]:.4f}
Loss Gap:   {final_loss_gap:.4f}

Train Acc:  {train_acc[-1]:.4f}
Val Acc:    {val_acc[-1]:.4f}
Acc Gap:    {final_acc_gap:.4f}

Epochs: {len(train_loss)}
    """
    
    plt.text(0.1, 0.5, text, fontsize=11, verticalalignment='center',
             bbox=dict(boxstyle='round', facecolor=color, alpha=0.3),
             family='monospace')
    
    plt.suptitle('🔍 Analyse Complète de l\'Overfitting', 
                 fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"📊 Analyse sauvegardée: {save_path}")
    plt.show()

# Créer la visualisation complète
plot_overfitting_analysis(history, save_path=f'{model_name}_overfitting.png')

-------------------------------------------------------------------------------
Standard Training Visualisation
-

In [ ]:
# Visualiser l'historique d'entraînement
plot_training_history(history, save_path=f'{model_name}_history.png')

# Afficher les métriques finales
print("\n" + "="*60)
print("Métriques d'entraînement")
print("="*60)
print(f"Meilleure val_accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"Meilleure val_loss: {min(history.history['val_loss']):.4f}")
print(f"Epoch final: {len(history.history['loss'])}")
print("="*60)

--------------
Evaluation on test set
-

In [ ]:
print("\n" + "="*60)
print("ÉVALUATION SUR LE TEST SET")
print("="*60)

# Évaluation
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

print(f"\n📊 Résultats sur le test set:")
print(f"   Test Loss:     {test_loss:.4f}")
print(f"   Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

# Prédictions
print("\n🔮 Génération des prédictions...")
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# Classification report
print("\n" + "="*60)
print("Classification Report")
print("="*60)
print(classification_report(y_test, y_pred, target_names=[str(i) for i in range(num_classes)]))

# Matrice de confusion
plot_confusion_matrix(
    y_test, y_pred, 
    classes=list(range(num_classes)),
    save_path=f'{model_name}_confusion_matrix.png'
)

print("="*60)

-----------------------
Saving of final model
-

In [ ]:
# Sauvegarder le modèle final
final_model_path = f'{model_name}_final.keras'
model.save(final_model_path)

print("\n" + "="*60)
print("SAUVEGARDE DU MODÈLE")
print("="*60)
print(f"✅ Modèle final sauvegardé: {final_model_path}")
print(f"✅ Meilleur modèle sauvegardé: {model_name}_best.keras")
print("="*60)

# Pour charger le modèle plus tard:
print("\nPour charger le modèle:")
print(f"  model = keras.models.load_model('{final_model_path}')")

---------------------------
Test on testing samples
-

In [ ]:
# Exemple de prédiction sur quelques échantillons
n_samples = 10
sample_indices = np.random.choice(len(X_test), n_samples, replace=False)

print("\n" + "="*60)
print(f"PRÉDICTIONS SUR {n_samples} ÉCHANTILLONS ALÉATOIRES")
print("="*60)

for i, idx in enumerate(sample_indices):
    sample = X_test[idx:idx+1]  # Shape: (1, sequence_length, 1)
    true_label = y_test[idx]
    
    # Prédiction
    pred_proba = model.predict(sample, verbose=0)[0]
    pred_label = np.argmax(pred_proba)
    confidence = pred_proba[pred_label]
    
    # Afficher
    status = "✅" if pred_label == true_label else "❌"
    print(f"\n{status} Échantillon {i+1}:")
    print(f"   Vraie classe:   {true_label}")
    print(f"   Classe prédite: {pred_label}")
    print(f"   Confiance:      {confidence:.4f} ({confidence*100:.2f}%)")
    
    # Top 3 prédictions
    top3_idx = np.argsort(pred_proba)[-3:][::-1]
    print(f"   Top 3: {[(idx, f'{pred_proba[idx]:.3f}') for idx in top3_idx]}")

print("\n" + "="*60)

--------------------------------
Final resume and Instructions
-

In [ ]:
print("\n" + "="*70)
print("                    🎉 RÉSUMÉ FINAL 🎉")
print("="*70)
print(f"\n✅ Modèle entraîné avec succès!")
print(f"✅ Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"\n📁 Fichiers générés:")
print(f"   - {model_name}_best.keras (meilleur modèle)")
print(f"   - {model_name}_final.keras (modèle final)")
print(f"   - {model_name}_history.png (courbes d'apprentissage)")
print(f"   - {model_name}_confusion_matrix.png (matrice de confusion)")
print(f"   - ./logs/{model_name}/ (logs TensorBoard)")

print(f"\n📊 Pour visualiser avec TensorBoard:")
print(f"   tensorboard --logdir=./logs/{model_name}")
print(f"   Puis ouvrir: http://localhost:6006")

print(f"\n🔮 Pour utiliser le modèle plus tard:")
print(f"   from tensorflow import keras")
print(f"   model = keras.models.load_model('{model_name}_best.keras')")
print(f"   predictions = model.predict(new_data)")

print("\n" + "="*70)